In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 4 on 12 virtual cores
Environment:
  JULIA_NUM_THREADS = 4


## 9-1. タスクとチャネル

### 9-1-1. タスク

#### コード9-1. 実行に5秒かかるコード例

In [2]:
begin
    sleep(5)
    print("Done.")
end

Done.

#### コード9-2. タスクの定義例

In [3]:
@task begin
    sleep(5)
    print("Done.")
end

Task (runnable) @0x00007fe8fe020740

#### コード9-3. タスクの実行例

In [4]:
t = @task begin
    sleep(5)
    print("Done.")
end

Task (runnable) @0x00007fe8fe9f8b90

In [5]:
schedule(t)

Task (runnable) @0x00007fe8fe9f8b90

Done.

#### コード9-4. `@async` の例

In [6]:
@async begin
    sleep(5)
    print("Done.")
end

Task (runnable) @0x00007fe8fe9f8e70

Done.

#### コード9-5. `wait()` の例

In [7]:
t = @async begin
    sleep(5)
    print("Done.")
end

Task (runnable) @0x00007fe8fe9f8fe0

In [8]:
wait(t)  # ←5秒以内に実行すること！
# ↓しばらくしてから表示される
# Done.
# ↑これが表示されるまではプロンプトが返ってこない

Done.

In [9]:
t1 = @task begin
    println("t1 start.")
    sleep(3)
    println("t1 end.")
end;

t2 = @task begin
    println("t2 start.")
    wait(t1)
    println("t2 end.")
end;

schedule(t1); schedule(t2); wait(t2)

t1 start.
t2 start.
t1 end.
t2 end.


#### コード9-6. `@sync` の例

In [10]:
@sync begin
    t1 = @async begin
        println("t1 start.")
        sleep(3)
        println("t1 end.")
    end
    @async begin
        println("t2 start.")
        wait(t1)
        println("t2 end.")
    end
end;

t1 start.
t2 start.
t1 end.
t2 end.


#### コード9-7. `fetch()` の例

In [11]:
begin
    t1 = @async begin
        println("t1 start.")
        sleep(3)
        "t1 end."
    end
    t2 = @async begin
        println("t2 start.")
        msg = fetch(t1)
        println(msg)
        "t2 end."
    end
    println(fetch(t2))
end

t1 start.
t2 start.
t1 end.
t2 end.


In [12]:
fetch(t1)

"t1 end."

In [13]:
fetch(t2)

"t2 end."

### 9-1-2. 通知

#### コード9-8. 通知による協調動作の例

In [14]:
cond1 = Condition();

cond2 = Condition();

@sync begin
    @async begin
        wait(cond2)
        println("t1 start.")
        notify(cond1)
        wait(cond2)
        println("t1 end.")
        notify(cond1)
    end
    @async begin
        wait(cond1)
        println("t2 start.")
        notify(cond2)
        wait(cond1)
        println("t2 end.")
        notify(cond2)
    end
    @async notify(cond2)
end;

t1 start.
t2 start.
t1 end.
t2 end.


### 9-1-3. チャネル

#### コード9-9. `Channel` の使用例(1)

In [15]:
chnl = Channel()

Channel{Any}(0) (empty)

In [16]:
isempty(chnl)

true

In [17]:
t = @async begin
    put!(chnl, "task start")
    n = 1
    while true
        put!(chnl, n)
        n += 1
        n > 15 && break
    end
    put!(chnl, "task end")
end

Task (runnable) @0x00007fe8685448b0

In [18]:
isempty(chnl)  # 1件でも `put!(～)` で追加した要素がある

false

In [19]:
take!(chnl)

"task start"

In [20]:
take!(chnl)

1

In [21]:
take!(chnl)

2

In [22]:
take!(chnl);
take!(chnl);
take!(chnl);
take!(chnl);
take!(chnl);
take!(chnl);
take!(chnl);
take!(chnl);
take!(chnl);
take!(chnl);
take!(chnl);
take!(chnl);

In [23]:
take!(chnl)

15

In [24]:
take!(chnl)

"task end"

In [25]:
isempty(chnl)  # 全部列挙し終わった（もう `put!()` で追加した要素がない）

true

In [ ]:
# take!(chnl)  # ※注意：返ってこないので実行しないこと！

In [26]:
close(chnl)

In [27]:
isempty(chnl)

true

In [28]:
take!(chnl)

LoadError: InvalidStateException: Channel is closed.

#### コード9-10. `Channel` の使用例(1) の動作確認用コード

In [29]:
function checkchannel()
    chnl = Channel()
    t = @async begin
        println("putting value...")
        put!(chnl, "task start")
        println("put value: ", "task start")
        n = 1
        while true
            println("putting value...")
            put!(chnl, n)
            println("put value: ", n)
            n += 1
            n > 15 && break
        end
        println("putting value...")
        put!(chnl, "task end")
        println("put value: ", "task end")
    end

    while true
        println("taking value...")
        v = take!(chnl)
        println("taken value: ", v)
        v == "task end" && break
    end
    wait(t)
    close(chnl)
end

checkchannel (generic function with 1 method)

#### コード9-11. `Channel` の使用例(1) の動作確認

In [30]:
checkchannel()

taking value...
putting value...
taken value: task start
taking value...
put value: task start
putting value...
taken value: 1
taking value...
put value: 1
putting value...
taken value: 2
taking value...
put value: 2
putting value...
taken value: 3
taking value...
put value: 3
putting value...
taken value: 4
taking value...
put value: 4
putting value...
taken value: 5
taking value...
put value: 5
putting value...
taken value: 6
taking value...
put value: 6
putting value...
taken value: 7
taking value...
put value: 7
putting value...
taken value: 8
taking value...
put value: 8
putting value...
taken value: 9
taking value...
put value: 9
putting value...
taken value: 10
taking value...
put value: 10
putting value...
taken value: 11
taking value...
put value: 11
putting value...
taken value: 12
taking value...
put value: 12
putting value...
taken value: 13
taking value...
put value: 13
putting value...
taken value: 14
taking value...
put value: 14
putting value...
taken value: 15
taking v

#### コード9-12. `Channel` の使用例(2): Doブロック構文 の利用

In [31]:
chnl = Channel() do chnl
    put!(chnl, "task start")
    n = 1
    while true
        put!(chnl, n)
        n += 1
        n > 15 && break
    end
    put!(chnl, "task end")
end

Channel{Any}(0) (1 item available)

In [32]:
for n in chnl
    println(n)
end

task start
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
task end


In [33]:
take!(chnl)

LoadError: InvalidStateException: Channel is closed.

#### コード9-13. `Channel` の使用例(3): 要素の型の指定

In [34]:
function collatz_channel(n)
    Channel{Int}() do chnl
        put!(chnl, n)
        while n != 1
            n = iseven(n) ? n ÷ 2 : 3n + 1
            put!(chnl, n)
        end
    end
end

collatz_channel (generic function with 1 method)

In [35]:
typeof(collatz_channel(3))

Channel{Int64}

In [36]:
eltype(collatz_channel(3))

Int64

In [37]:
for n in collatz_channel(3)
    println(n)
end

3
10
5
16
8
4
2
1


In [38]:
collect(collatz_channel(27))

112-element Vector{Int64}:
  27
  82
  41
 124
  62
  31
  94
  47
 142
  71
 214
 107
 322
   ⋮
  53
 160
  80
  40
  20
  10
   5
  16
   8
   4
   2
   1

#### コード9-14. `Channel` の使用例(4) バッファドチャネル

In [39]:
chnl = Channel(3) do chnl
    put!(chnl, "task start")
    n = 1
    while true
        put!(chnl, n)
        n += 1
        n > 15 && break
    end
    put!(chnl, "task end")
end

Channel{Any}(3) (4 items available)

In [40]:
for v in chnl
    println(v)
end

task start
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
task end


#### コード9-15. `Channel` の使用例(4) の動作確認用コード

In [41]:
function checkchannelB()
    chnl = Channel(3)
    t = @async begin
        println("putting value...")
        put!(chnl, "task start")
        println("put value: ", "task start")
        n = 1
        while true
            println("putting value...")
            put!(chnl, n)
            println("put value: ", n)
            n += 1
            n > 15 && break
        end
        println("putting value...")
        put!(chnl, "task end")
        println("put value: ", "task end")
    end

    while true
        println("taking value...")
        v = take!(chnl)
        println("taken value: ", v)
        v == "task end" && break
    end
    wait(t)
    close(chnl)
end


checkchannelB (generic function with 1 method)

#### コード9-16. `Channel` の使用例(4) の動作確認

In [42]:
checkchannelB()

taking value...
putting value...
put value: task start
putting value...
put value: 1
putting value...
put value: 2
putting value...
put value: 3
putting value...
taken value: task start
taking value...
taken value: 1
taking value...
taken value: 2
taking value...
taken value: 3
taking value...
taken value: 4
taking value...
put value: 4
putting value...
put value: 5
putting value...
put value: 6
putting value...
put value: 7
putting value...
put value: 8
putting value...
taken value: 5
taking value...
taken value: 6
taking value...
taken value: 7
taking value...
taken value: 8
taking value...
taken value: 9
taking value...
put value: 9
putting value...
put value: 10
putting value...
put value: 11
putting value...
put value: 12
putting value...
put value: 13
putting value...
taken value: 10
taking value...
taken value: 11
taking value...
taken value: 12
taking value...
taken value: 13
taking value...
taken value: 14
taking value...
put value: 14
putting value...
put value: 15
putting va

#### コード9-17. `fetch(chnl)` の例

In [43]:
chnl = Channel(3) do chnl
    put!(chnl, 1)
    put!(chnl, 2)
    put!(chnl, 3)
end

Channel{Any}(3) (closed)

In [44]:
fetch(chnl)

1

In [45]:
fetch(chnl)

1

In [46]:
take!(chnl)

1

In [47]:
fetch(chnl)

2

In [48]:
take!(chnl)

2

In [49]:
take!(chnl)

3

In [50]:
fetch(chnl)  # もしくは `take!(chnl)`

LoadError: InvalidStateException: Channel is closed.

In [51]:
fetch(Channel())  # アンバッファドチャネルを渡した場合

LoadError: `fetch` is not supported on an unbuffered Channel.

### 9-1-4. タスク・チャネル による並行処理

#### コード9-18. `SearchJuliaSample.jl`（`Channel` の実用例（並行処理の例））

In [52]:
module SearchJuliaSample

using Downloads

const UA = "Mozilla/5.0 (Julia v$(VERSION))"

struct SearchEngine <: Function
    name::String
    url::String
end
const Google = SearchEngine("Google", "https://www.google.com/search?q=julialang&oe=UTF-8")
const Bing = SearchEngine("Bing", "https://www.bing.com/search?q=julialang")
const DuckDuckGo = SearchEngine("DuckDuckGo", "https://duckduckgo.com/?q=julialang")

struct SearchSummary{R}
    engine::String
    exec_time::Float64
    response::R  # Maybe Downloads.Response
    body::Vector{UInt8}
end
function Base.show(io::IO, summary::SearchSummary)
    println(io, "engine: ", summary.engine)
    println(io, "exec_time: ", summary.exec_time)
    print(io, "response: (")
    print(io, "$(summary.response.status), ")
    print(io, "$(length(summary.response.headers)) headers, ")
    println(io, "$(length(summary.body)) bytes in body)")
end

function (engine::SearchEngine)()
    buf = IOBuffer()
    res, exec_time = @timed Downloads.request(
        engine.url;
        headers=["User-Agent"=>UA],
        output=buf
    )
    SearchSummary(engine.name, exec_time, res, take!(buf))
end
function (engine::SearchEngine)(channel)
    summary = engine()
    if isopen(channel)
        put!(channel, summary)
    end
end

# "fcfs" stands for "first come, first served"  # 日本語で「早い者勝ち」
function fcfs()
    results = Channel{SearchSummary{Downloads.Response}}()

    @async Google(results)
    @async Bing(results)
    @async DuckDuckGo(results)

    res0 = take!(results)  # 同期
    close(results)

    print(res0)
end

function race()
    results = Channel{SearchSummary{Downloads.Response}}(3)

    @async Google(results)
    @async Bing(results)
    @async DuckDuckGo(results)

    res1 = take!(results)  # 同期
    println(res1)

    res2 = take!(results)  # 同期
    println(res2)

    res3 = take!(results)  # 同期
    print(res3)

    close(results)
end

end

Main.SearchJuliaSample

#### コード9-19. `SearchJuliaSample.jl` の実行例(1) 基本動作

In [53]:
SearchJuliaSample.Google()

engine: Google
exec_time: 0.554297135
response: (200, 15 headers, 73248 bytes in body)


In [54]:
SearchJuliaSample.Bing()

engine: Bing
exec_time: 0.732116174
response: (200, 20 headers, 394873 bytes in body)


In [55]:
SearchJuliaSample.DuckDuckGo()

engine: DuckDuckGo
exec_time: 0.535617537
response: (200, 17 headers, 24392 bytes in body)


#### コード9-20. `SearchJuliaSample.jl` の実行例(2) 並行処理

In [56]:
SearchJuliaSample.fcfs()

engine: DuckDuckGo
exec_time: 0.186248922
response: (200, 17 headers, 24392 bytes in body)


In [57]:
SearchJuliaSample.fcfs()

engine: DuckDuckGo
exec_time: 0.089702679
response: (200, 17 headers, 24392 bytes in body)


In [58]:
SearchJuliaSample.fcfs()

engine: DuckDuckGo
exec_time: 0.097099061
response: (200, 17 headers, 24392 bytes in body)


In [59]:
SearchJuliaSample.race()

engine: DuckDuckGo
exec_time: 0.18229694
response: (200, 17 headers, 24392 bytes in body)

engine: Google
exec_time: 0.358032334
response: (200, 15 headers, 73074 bytes in body)

engine: Bing
exec_time: 0.550269116
response: (200, 20 headers, 421951 bytes in body)


In [60]:
SearchJuliaSample.race()

engine: DuckDuckGo
exec_time: 0.186398376
response: (200, 17 headers, 24392 bytes in body)

engine: Bing
exec_time: 0.45394427
response: (200, 20 headers, 406376 bytes in body)

engine: Google
exec_time: 0.570814537
response: (200, 15 headers, 73448 bytes in body)
